<a href="https://colab.research.google.com/github/pradipNP/8x-Puzzle-Solver-Using-A-Star-Search/blob/main/8xPuzzle_A_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import defaultdict
import heapq
from copy import deepcopy

class PuzzleState:
    def __init__(self, board, parent=None, move=None, depth=0):
        self.board = board
        self.parent = parent
        self.move = move
        self.depth = depth

        # Find position of empty tile (0)
        for i in range(3):
            for j in range(3):
                if board[i][j] == 0:
                    self.empty = (i, j)
                    break

    def get_neighbors(self):
        neighbors = []
        moves = [
            ("Up", (-1, 0)),
            ("Down", (1, 0)),
            ("Left", (0, -1)),
            ("Right", (0, 1))
        ]

        for move_name, (dx, dy) in moves:
            new_x, new_y = self.empty[0] + dx, self.empty[1] + dy

            if 0 <= new_x < 3 and 0 <= new_y < 3:
                new_board = deepcopy(self.board)
                # Swap empty tile with neighbor
                new_board[self.empty[0]][self.empty[1]] = new_board[new_x][new_y]
                new_board[new_x][new_y] = 0
                neighbors.append(PuzzleState(new_board, self, move_name, self.depth + 1))

        return neighbors

    def misplaced_tiles(self, goal):
        count = 0
        for i in range(3):
            for j in range(3):
                if self.board[i][j] != 0 and self.board[i][j] != goal[i][j]:
                    count += 1
        return count

    def manhattan_distance(self, goal):
        distance = 0
        goal_positions = {}

        # Create a map of value to position in goal state
        for i in range(3):
            for j in range(3):
                if goal[i][j] != 0:
                    goal_positions[goal[i][j]] = (i, j)

        # Calculate Manhattan distance for each tile
        for i in range(3):
            for j in range(3):
                if self.board[i][j] != 0:
                    goal_pos = goal_positions[self.board[i][j]]
                    distance += abs(i - goal_pos[0]) + abs(j - goal_pos[1])

        return distance

    def __lt__(self, other):
        return False  # Required for heapq with custom objects

def solve_puzzle(initial_state, goal_state, heuristic_type="manhattan"):
    start = PuzzleState(initial_state)

    # Priority queue for frontier
    frontier = []
    # Set for explored states
    explored = set()

    # Initial state
    heapq.heappush(frontier, (0, start))
    nodes_explored = 0

    while frontier:
        _, current = heapq.heappop(frontier)
        nodes_explored += 1

        # Check if goal reached
        if current.board == goal_state:
            return current, nodes_explored

        # Convert current board to tuple for hashing
        board_tuple = tuple(map(tuple, current.board))
        if board_tuple in explored:
            continue

        explored.add(board_tuple)

        # Generate and evaluate neighbors
        for neighbor in current.get_neighbors():
            if tuple(map(tuple, neighbor.board)) not in explored:
                # Calculate heuristic value
                h = (neighbor.manhattan_distance(goal_state) if heuristic_type == "manhattan"
                     else neighbor.misplaced_tiles(goal_state))
                f = neighbor.depth + h  # f = g + h
                heapq.heappush(frontier, (f, neighbor))

    return None, nodes_explored

def print_solution_path(solution):
    if not solution:
        return

    path = []
    current = solution
    while current:
        path.append((current.board, current.move))
        current = current.parent

    path.reverse()

    print("\nSolution Path:")
    for idx, (board, move) in enumerate(path):
        print(f"\nStep {idx} {f'(Move: {move})' if move else '(Initial State)'}")
        for row in board:
            print(row)

# Test the implementation
initial_state = [
    [1, 6, 2],
    [4, 0, 3],
    [7, 5, 8]
]

goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

# Test with both heuristics
for heuristic in ["manhattan", "misplaced"]:
    print(f"\nSolving with {heuristic} heuristic...")
    solution, nodes = solve_puzzle(initial_state, goal_state, heuristic)

    if solution:
        print(f"Solution found in {solution.depth} moves")
        print(f"Nodes explored: {nodes}")
        print_solution_path(solution)
    else:
        print("No solution found!")



Solving with manhattan heuristic...
Solution found in 6 moves
Nodes explored: 9

Solution Path:

Step 0 (Initial State)
[1, 6, 2]
[4, 0, 3]
[7, 5, 8]

Step 1 (Move: Up)
[1, 0, 2]
[4, 6, 3]
[7, 5, 8]

Step 2 (Move: Right)
[1, 2, 0]
[4, 6, 3]
[7, 5, 8]

Step 3 (Move: Down)
[1, 2, 3]
[4, 6, 0]
[7, 5, 8]

Step 4 (Move: Left)
[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

Step 5 (Move: Down)
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

Step 6 (Move: Right)
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

Solving with misplaced heuristic...
Solution found in 6 moves
Nodes explored: 11

Solution Path:

Step 0 (Initial State)
[1, 6, 2]
[4, 0, 3]
[7, 5, 8]

Step 1 (Move: Up)
[1, 0, 2]
[4, 6, 3]
[7, 5, 8]

Step 2 (Move: Right)
[1, 2, 0]
[4, 6, 3]
[7, 5, 8]

Step 3 (Move: Down)
[1, 2, 3]
[4, 6, 0]
[7, 5, 8]

Step 4 (Move: Left)
[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

Step 5 (Move: Down)
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

Step 6 (Move: Right)
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]
